In [ ]:
# here put the import lib
import os
import sys
from collections import defaultdict
import random
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataloader
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(""))))
from utils.data_utils import data_utils, data_process
from utils.fetch_features import *
from utils.torch_utils import torch_utils
from models.baseline_mlp import baseline_lstm

random.seed(35)
plt.style.use("seaborn")
%matplotlib inline

print("torch.version:  ", torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("current_device: ", device)

if torch.cuda.is_available():
    print("device_name:    ", torch.cuda.get_device_name(device))
    # print("device_count:   ", torch.cuda.device_count())
    # print("current_device: ", torch.cuda.current_device())
    torch.cuda.empty_cache()

# read action_state

In [ ]:
def trans_2_symbols(action_condition):
    def generate_symbol_index(action_state_var):
        sym = []
        for symbol_variable in action_state_var.values():
            for symbol in symbol_variable:
                sym.append(symbol)

        return "symbol{}".format(len(set(sym)))

    action_state_var = defaultdict(list)
    for action in ["move", "pick_cube", "transport", "place_cube"]:
        symbol_varibales = {}
        for symbol_data in action_condition[action]:
            new_index = generate_symbol_index(action_state_var)
            symbol_varibales[new_index] = symbol_data
            action_state_var[action] = symbol_varibales

    return action_state_var


with open("eff_pos.json", "r") as f:
    content = json.load(f)

action_state = trans_2_symbols(content)

# with open("action_state_variable.json", "w") as f:
#     json.dump(action_state, f)

# read data

In [ ]:
# -----------------------------
# read segmented trajectory data
# -----------------------------


def read_train_data(*file_names):
    traj_list = []
    for file_name in file_names:
        temp = data_utils.read_from_pickle(
            os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "archive", "processed", file_name)
        )
        traj_list.extend(temp)
    return traj_list


traj_list = read_train_data("traj_stationary_65d.pickle")


def compute_mean_std(traj_list):
    data_dict = {}
    for action in FETCH_CAPABILITY_BASE:
        data_dict.setdefault(action, [])

    for traj in traj_list:
        for action in traj:
            data_dict[action].append(traj[action])

    z_score_mean, z_score_std = data_process.compute_minibatch_mean_and_std(data_dict)
    return z_score_mean, z_score_std


# compute mean std
z_score_mean, z_score_std = compute_mean_std(traj_list)
np.save("baseline_lstm_mean.npy", z_score_mean)
np.save("baseline_lstm_std.npy", z_score_std)



# Generate train_loader, test_loader

In [ ]:
z_score_mean = np.load("baseline_lstm_mean.npy")
z_score_std = np.load("baseline_lstm_std.npy")


def z_score_normalization(x, mean, std):
    return (x - mean) / (std + 1e-5)


class Collate_RNN:
    def __init__(self):
        pass

    def _collate(self, batch):

        xs = [torch.FloatTensor(v[0]).to(device) for v in batch]
        ys = torch.LongTensor([v[1] for v in batch])
        # 获得每个样本的序列长度
        seq_lengths = torch.LongTensor([v for v in map(len, xs)])
        max_len = max([len(v) for v in xs])
        # 每个样本都padding到当前batch的最大长度
        xs = pad_sequence(xs, batch_first=True, padding_value=0)
        # 把xs和ys按照序列长度从大到小排序
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        xs = xs[perm_idx]
        ys = ys[perm_idx]
        return xs, seq_lengths, ys

    def __call__(self, batch):
        return self._collate(batch)


def generate_training_dataset(action, symbol_states, traj_list, z_score_mean, z_score_std, time_step):
    """
    @description  :  generate training dataset for action
    ---------
    @param  :
    ---------
    @Returns  :  X y
    ---------
    """

    train_data = []
    train_label = []
    for _1_traj_data in traj_list:
        for index in range(350, len(_1_traj_data[action][:-350]), 12):
            if (index + time_step) < (len(_1_traj_data[action]) - 350):
                train_data.append(np.array(_1_traj_data[action][index : index + time_step]))
                train_label.append(0)
        for index in range(len(_1_traj_data[action]) - 350, len(_1_traj_data[action]), 1):
            if (index + time_step) < len(_1_traj_data[action]):
                train_data.append(np.array(_1_traj_data[action][index : index + time_step]))
                train_label.append(1)

    label_0 = 0
    label_1 = 0
    for label in train_label:
        if label == 0:
            label_0 = label_0 + 1
        elif label == 1:
            label_1 = label_1 + 1
    print("    Num of label 1: ", label_1)
    print("    Num of label 0: ", label_0)

    X = []
    for x in train_data:
        X.append(
            data_process.select_features(
                data_process.z_score_normalization(np.array(x).astype(float), z_score_mean, z_score_std), symbol_states
            )  # .flatten()
        )
    X = np.array(X)
    y = np.array(train_label)
    return X, y

In [ ]:
def train_net(model, train_loader, test_loader):

    train_losses = []
    test_losses = []
    avg_train_losses = []
    avg_test_losses = []
    test_accuracy = []
    avg_test_accuracy = []

    LR = 0.0003
    EPOCH = 100
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=0.1)
    loss_func = nn.CrossEntropyLoss()
    _lambda = lambda epoch: 0.999 ** epoch
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=_lambda)

    for epoch in range(EPOCH):

        for step, (train_x, lengths, train_y) in enumerate(train_loader):

            train_x = train_x.to(device)
            train_y = train_y.to(device)
            # train_y = train_y.unsqueeze(1)

            # print('train_y: ', train_y.size(), train_y)
            model.train()
            train_x = pack_padded_sequence(train_x, lengths, batch_first=True)
            output = model(train_x)

            # print('output: ', output.size(), output)
            # print(output.size())
            loss = loss_func(output, train_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            if step % 50 == 0:

                if torch.cuda.is_available():
                    train_losses.append(loss.data.cpu().numpy())
                else:
                    train_losses.append(loss.data.numpy())

                i = 0
                total_size = 0

                with torch.no_grad():
                    for _, (test_x, lengths, test_y) in enumerate(test_loader):
                        test_x = test_x.to(device)
                        test_y = test_y.to(device)
                        # test_y = test_y.unsqueeze(1)
                        model.eval()
                        test_x = pack_padded_sequence(test_x, lengths, batch_first=True)
                        test_output = model(test_x)
                        loss = loss_func(test_output, test_y)
                        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()

                        for res in test_y.cpu().numpy() == pred_y:
                            if res:
                                i = i + 1
                        total_size = total_size + test_output.size()[0]

                    accuracy = i / total_size
                    test_accuracy.append(accuracy)

                    if torch.cuda.is_available():
                        test_losses.append(loss.data.cpu().numpy())
                    else:
                        test_losses.append(loss.data.numpy())

        avg_train_loss = np.average(train_losses)
        avg_test_loss = np.average(test_losses)
        avg_train_losses.append(avg_train_loss)
        avg_test_losses.append(avg_test_loss)
        avg_accuracy = np.average(test_accuracy)
        avg_test_accuracy.append(avg_accuracy)

        if (epoch + 1) % 10 == 0:
            print_msg = "Epoch: {}/{} | train loss: {:.4f} | test loss: {:.4f} | test accuracy: {:.4f}".format(
                epoch + 1,
                EPOCH,
                avg_train_loss,
                avg_test_loss,
                avg_accuracy,
            )

            print(print_msg)

        # if torch_utils.early_stop(model=model, test_accuracy=avg_test_accuracy):
        #     print("Early Stopping")
        #     break

        train_losses = []
        test_losses = []
        test_accuracy = []

    # model.load_state_dict(torch.load("checkpoint.pt"))
    return model, avg_train_losses, avg_test_losses

In [ ]:
# start trainning

BATCH_SIZE = 256
TIME_STEP = 200

for action in action_state:
    print(action)
    for symbol in action_state[action]:
        print(" ", symbol)

        # 初始化symbol dataset
        X, y = generate_training_dataset(
            action, action_state[action][symbol], traj_list, z_score_mean, z_score_std, time_step=TIME_STEP
        )

        x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
        train_loader = dataloader.DataLoader(
            dataset=list(zip(x_train, y_train)), batch_size=BATCH_SIZE, num_workers=0, shuffle=True, collate_fn=Collate_RNN()
        )
        test_loader = dataloader.DataLoader(
            dataset=list(zip(x_test, y_test)), batch_size=BATCH_SIZE, num_workers=0, shuffle=True, collate_fn=Collate_RNN()
        )

        n_features = len(action_state[action][symbol])
        model = baseline_lstm(n_features)
        model.to(device)
        # print(model)

        print("    Start Train ...")
        model, avg_train_losses, avg_test_losses = train_net(model, train_loader, test_loader)
        torch.save(model.state_dict(), "models/{}.pt".format(symbol))
        print("    Training Finish")
        print()